In [10]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../../examples/actuation_sparsification/half_vase_opened.msh')
mat = elastic_rods.RodMaterial('+', 20000, 0.3, [2.0, 2.0, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

torques = np.loadtxt('data/half_vase/torques_0.122069_L0.5_0.txt')
externalForces = np.zeros(l.numDoF())
externalForces[l.jointAngleDoFIndices()] = torques

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, l.averageJointAngle, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [11]:
with suppress_stdout(): elastic_rods.compute_equilibrium(l, 1.0 * externalForces, fixedVars=fixedVars)
view.update()

In [9]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_half_vase_actuated.txt', zeroBasedIndexing=True)
np.savetxt('restlen_half_vase_actuated.txt',l.getPerSegmentRestLength())
np.savetxt('normals_half_vase_actuated', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_half_vase_actuated_opened.msh')

In [4]:
print('\n'.join(map(str, np.where(torques > 1e-5)[0])), file=open('half_vase_actuated_joints.txt', 'w'))